In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import multiprocessing
import shutil
from pathlib import Path
import numpy as np
import thor.data as data
import thor.parallel as parallel
import thor.visualize as visualize
import thor.analyze as analyze
import thor.option as option
import thor.grid as grid

notebook_name = "parallel_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



# Parallel

In [2]:
 # Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T12:00"
end = "2005-11-13T20:00"

intervals = parallel.get_time_intervals(start, end)
output_parent = base_local / "runs/cpol_demo_parallel"
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

 # Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)

# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)

data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

data.dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

altitude = list(np.arange(0, 20e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="cpol")
track_options.to_yaml(options_directory / "track.yml")
visualize_options = None

2024-10-16 15:27:08,274 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/data.yml
2024-10-16 15:27:08,293 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-16 15:27:08,294 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-16 15:27:08,295 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/options/grid.yml


In [3]:
with multiprocessing.Pool(initializer=parallel.initialize_process()) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.model_copy(deep=True), visualize_options] 
        args += [output_parent, "cpol"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)

2024-10-16 15:29:38,886 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_2/options/data.yml
2024-10-16 15:29:38,902 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_1/options/data.yml
2024-10-16 15:29:38,912 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_0/options/data.yml
2024-10-16 15:29:38,952 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_2/options/grid.yml
2024-10-16 15:29:38,965 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_7/options/data.yml
2024-10-16 15:29:38,969 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_4/options/data.yml
2024-10-16 15:29:38,975 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_parallel/interval_0/options/grid.yml
2024-10-16 15

In [5]:
parallel.stitch_run(output_parent, intervals)

2024-10-16 15:32:44,878 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-10-16 15:32:46,049 - thor.parallel - INFO - Stitching record files.
2024-10-16 15:32:46,098 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/records/filepaths/cpol.csv
2024-10-16 15:32:46,101 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/records/filepaths/cpol.yml
2024-10-16 15:32:46,103 - thor.parallel - INFO - Stitching attribute files.
2024-10-16 15:32:46,219 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/anvil/core.csv
2024-10-16 15:32:46,221 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/attributes/mcs/anvil/core.yml
2024-10-16 15:32:46,322 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_

# Analyze

In [6]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-16 15:32:52,925 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/velocities.csv
2024-10-16 15:32:52,929 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/velocities.yml
2024-10-16 15:32:53,116 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/quality.csv
2024-10-16 15:32:53,118 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/quality.yml
2024-10-16 15:32:53,183 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/classification.csv
2024-10-16 15:32:53,185 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_parallel/analysis/classification.yml


,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 14:10:23,1,trailing,left,left,shear-perpendicular,up-shear
2005-11-13 14:20:09,1,trailing,left,left,up-shear,up-shear
2005-11-13 14:30:09,1,trailing,left,left,up-shear,up-shear
2005-11-13 14:40:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 14:50:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 15:00:08,1,trailing,front,left,up-shear,shear-perpendicular
2005-11-13 15:10:23,1,trailing,front,trailing,up-shear,shear-perpendicular
2005-11-13 15:20:09,1,trailing,front,trailing,up-shear,shear-perpendicular
2005-11-13 15:30:09,1,trailing,front,trailing,up-shear,down-shear


In [7]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation"
)
start_time = np.datetime64("2005-11-13T12:00")
end_time = np.datetime64("2005-11-13T22:00")
visualize.attribute.mcs_series(
    output_parent, start_time, end_time, figure_options, parallel_figure=True
)

2024-10-16 15:32:57,239 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.120000.nc
2024-10-16 15:32:57,301 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-16 15:33:00,227 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2005-11-13T12:00:09.000000000.
2024-10-16 15:33:00,228 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time 2005-11-13T12:00:09.000000000.
2024-10-16 15:33:00,327 - thor.visualize.attribute - DEBUG - Creating grouped mask figure at time 2005-11-13T12:00:09.000000000.
2024-10-16 15:33:00,328 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-16 15:33:00,455 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-16 15:33:00,465 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-16 15:33:00,503 - thor.visualize.attribute - DEBUG - Saving mcs_velocity_analysis figure for 2005-11-13T12:00:09.000000000.
20